In [12]:
""" Example Test: 
1) get an datajoint neurite
2) get how many branch points greater than 2
3) Find the distribution of distances between them
4) Pick good threshold
5) Run flagging analysis on a couple and see if can pick up any merge errors

"""

' Example Test: \n1) get an datajoint neurite\n2) get how many branch points greater than 2\n3) Find the distribution of distances between them\n4) Pick good threshold\n5) Run flagging analysis on a couple and see if can pick up any merge errors\n\n'

In [1]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

In [3]:
#configure datajoint access to skeletons
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

In [4]:
ta3p100.FilteredSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,spine_filter_length,n_edges,edges,lengths_of_removed_limbs
2,648518346341351467,3000,517,=BLOB=,=BLOB=
2,648518346341351503,3000,323,=BLOB=,=BLOB=
2,648518346341351508,3000,486,=BLOB=,=BLOB=
2,648518346341351509,3000,348,=BLOB=,=BLOB=
2,648518346341351512,3000,576,=BLOB=,=BLOB=
2,648518346341351514,3000,558,=BLOB=,=BLOB=
2,648518346341351518,3000,391,=BLOB=,=BLOB=


In [5]:
ta3p100.NeuriteRawSkeleton()

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated,n_edges number of edges stored,edges array storing edges on each row,n_bodies the amount of segments the neurite was originally split into,lagest_mesh_perc the percentage of the entire mesh that the largest submesh makes up
2,648518346341351467,2019-03-29 20:06:00,526,=BLOB=,4,0.997983
2,648518346341351503,2019-03-29 19:59:45,338,=BLOB=,2,0.999375
2,648518346341351508,2019-03-29 19:59:57,486,=BLOB=,82,0.965987
2,648518346341351509,2019-03-29 20:00:01,365,=BLOB=,5,0.998435
2,648518346341351512,2019-03-29 20:00:07,612,=BLOB=,6,0.999543
2,648518346341351514,2019-03-29 20:00:13,595,=BLOB=,6,0.998915
2,648518346341351518,2019-03-29 20:00:17,403,=BLOB=,5,0.999109


In [90]:
ta3p100.LabelKey.fetch()

array([( 0, 'not_labeled', 'no_color'), ( 1, 'label_removed', 'no_color'),
       ( 2, 'Apical', 'blue'), ( 3, 'Basal', 'yellow'),
       ( 4, 'Oblique', 'green'), ( 5, 'Soma', 'red'),
       ( 6, 'Axon-Soma', 'aqua'), ( 7, 'Axon-Dendr', 'off blue'),
       ( 8, 'Dendrite', 'purple'), ( 9, 'Distal', 'pink'),
       (10, 'Error', 'brown'), (11, 'Unlabelable', 'tan'),
       (12, 'Cilia', 'light purple'), (13, 'Spine Head', 'rose'),
       (14, 'Spine', 'light pink'), (15, 'Spine Neck', 'light green'),
       (16, 'Bouton', 'aqua'), (17, 'shaft', 'dark orange'),
       (18, 'non-spine', 'light orange')],
      dtype=[('numeric', '<i8'), ('description', 'O'), ('color', 'O')])

In [6]:
#get all neurite skeletons
neurite_skeletons = ta3p100.NeuriteRawSkeleton.fetch(as_dict=True)

In [8]:
#just gets the ids and the edges
neurite_skeletons = (dj.U("segment_id","edges") & ta3p100.NeuriteRawSkeleton).fetch(as_dict=True)

In [11]:
example_skeleton = neurite_skeletons[0]
example_skeleton

OrderedDict([('segment_id', 648518346341351467),
             ('edges', array([[[234125. , 280734. ,  79143.5],
                      [234098. , 280572. ,  79063.4]],
              
                     [[234125. , 280734. ,  79143.5],
                      [234186. , 281010. ,  79266.6]],
              
                     [[214409. , 255227. ,  62211.9],
                      [214342. , 254958. ,  62507.7]],
              
                     ...,
              
                     [[211808. , 247227. ,  67090.7],
                      [211490. , 246776. ,  66933.2]],
              
                     [[229348. , 192774. ,  50601.6],
                      [229501. , 192716. ,  50250.7]],
              
                     [[206488. , 233182. ,  63286.1],
                      [206115. , 233287. ,  63122.9]]]))])

In [84]:
#get the unique number of nodes in skeelton
edges_unpacked  = example_skeleton["edges"].reshape(-1,3)
print(len(edges_unpacked))
unique_rows = np.unique(edges_unpacked, axis=0)
print(len(unique_rows))

#get a mapping of 

1052
527


In [74]:
unique_rows_list = unique_rows.tolist()
#unique_rows_list.index([234125. , 280734. ,  79143.5])

edges_unpacked_coefficients = [unique_rows_list.index(a) for a in edges_unpacked.tolist()]
edges_coefficients = np.array(edges_unpacked_coefficients).reshape(-1,2)
edges_coefficients

array([[483, 482],
       [483, 484],
       [258, 254],
       ...,
       [190, 187],
       [448, 450],
       [113, 100]])

In [75]:
#create the nodes with corresponding coordinates
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
#add the edges
B.add_edges_from(edges_coefficients)
B.number_of_nodes(), B.number_of_edges()

In [89]:
#check if any nodes have more than 2 degrees
possible_segmentations = [n for n,x in B.degree() if x > 2]
possible_segmentations

[74, 94, 113, 190, 243, 383]

In [79]:
#create a function for the filtered skeletons
ta3p100.FilteredSkeletonMinusSoma.fetch(as_dict=True)


(527, 526)

In [55]:
"""------------- exploration --------------------"""
x = np.array([[2,5,2],[4,6,4],[4,6,4]])
unique_x_axis_0 = np.unique(x,axis=0) #gets rid of rows that are same
unique_x_axis_0
unique_x_axis_1 = np.unique(x,axis=1) #gets rid of the columns that are same
unique_x_axis_1

array([[2, 5],
       [4, 6],
       [4, 6]])